In [1]:
import pyemma
import pyemma.coordinates as coor

import mdtraj as md
from pprint import pprint
import glob, os, shutil, subprocess
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tqdm
from collections import Counter

import deeptime as dt
from deeptime.markov.hmm import BayesianHMM
from deeptime.clustering import BoxDiscretization
from deeptime.clustering import RegularSpace


## Extract the data and save into data.pkl

In [6]:
import os
import shutil

# Define the directory path
directory = "omega_data/"
clean_dir = False

# Handle directory creation and cleanup
try:
    if clean_dir:
        if os.path.exists(directory):
            # Remove all contents of the directory
            shutil.rmtree(directory)
            print(f"All contents in '{directory}' have been cleared.")
        # Recreate the empty directory
        os.makedirs(directory, exist_ok=True)
        print(f"Directory '{directory}' has been created/recreated.")
    else:
        # Ensure the directory exists without cleaning
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(f"Directory '{directory}' did not exist, so it has been created.")
        else:
            print(f"Directory '{directory}' already exists. No action taken.")
except OSError as e:
    print(f"Error handling directory '{directory}': {e}")

Directory 'omega_data/' already exists. No action taken.


In [7]:
class Data:
    def __init__(self, array_list):
        self.array_list = array_list

    def save(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.array_list, f)

    @classmethod
    def load(cls, filename):
        with open(filename, 'rb') as f:
            array_list = pickle.load(f)
        return cls(array_list)

## Featurization

In [ ]:
peptoid_gro = 'single_peptoid.gro'
traj_dir = 'PROJ12463traj'

file_pattern = f"{traj_dir}/r*c*_inte.xtc"
files = glob.glob(file_pattern)
print(files)
feat = pyemma.coordinates.featurizer(peptoid_gro)

['PROJ12463traj/r1c5_inte.xtc', 'PROJ12463traj/r1c4_inte.xtc', 'PROJ12463traj/r0c32_inte.xtc', 'PROJ12463traj/r0c33_inte.xtc', 'PROJ12463traj/r7c69_inte.xtc', 'PROJ12463traj/r7c68_inte.xtc', 'PROJ12463traj/r3c93_inte.xtc', 'PROJ12463traj/r6c55_inte.xtc', 'PROJ12463traj/r6c54_inte.xtc', 'PROJ12463traj/r3c92_inte.xtc', 'PROJ12463traj/r5c66_inte.xtc', 'PROJ12463traj/r5c67_inte.xtc', 'PROJ12463traj/r3c48_inte.xtc', 'PROJ12463traj/r3c49_inte.xtc', 'PROJ12463traj/r4c13_inte.xtc', 'PROJ12463traj/r4c12_inte.xtc', 'PROJ12463traj/r1c47_inte.xtc', 'PROJ12463traj/r4c81_inte.xtc', 'PROJ12463traj/r4c80_inte.xtc', 'PROJ12463traj/r1c46_inte.xtc', 'PROJ12463traj/r2c74_inte.xtc', 'PROJ12463traj/r2c75_inte.xtc', 'PROJ12463traj/r6c9_inte.xtc', 'PROJ12463traj/r2c91_inte.xtc', 'PROJ12463traj/r6c8_inte.xtc', 'PROJ12463traj/r2c90_inte.xtc', 'PROJ12463traj/r4c64_inte.xtc', 'PROJ12463traj/r4c65_inte.xtc', 'PROJ12463traj/r1c30_inte.xtc', 'PROJ12463traj/r5c58_inte.xtc', 'PROJ12463traj/r1c31_inte.xtc', 'PROJ12463t

In [9]:
## Select feature 
CA_CA = feat.select(f'name == CA')

omega_atoms = [
    [6, 7, 18, 36],
    [36, 39, 41, 45],
    [45, 46, 57, 75]
]
# Convert to numpy array, subtract 1, and convert back to list (if required)
omega_atoms = np.array(omega_atoms) - 1
omega_atoms = omega_atoms.tolist()  


noe_atoms = [
    [16, 20],
    [16, 21],
    [17, 20],
    [17, 21],
    [37, 55],
    [37, 56],
    [38, 55],
    [38, 56]
]

# Convert to numpy array, subtract 1, and convert back to list (if required)
noe_atoms = np.array(noe_atoms) - 1
noe_atoms = noe_atoms.tolist()  

In [10]:
save_dir = 'omega_data'
feat.add_dihedrals(omega_atoms,deg=True) # Select the backbone omega angles 
np.save(f"{save_dir}/features", feat)
print(f"Features save at: {save_dir}/features.npy")

all_features = feat.describe()  # `describe()` should list all feature descriptions
for i, feature in enumerate(all_features):
    print(i+1, feature)

Features save at: omega_data/features.npy
1 DIH: Q03 2 CA 5 - Q03 2 C 6 - R07 3 N 17 - R07 3 CA 35 
2 DIH: R07 3 CA 35 - R07 3 C 38 - Q03 4 N 40 - Q03 4 CA 44 
3 DIH: Q03 4 CA 44 - Q03 4 C 45 - R07 5 N 56 - R07 5 CA 74 


In [11]:
data = coor.load(files, features=feat)
print('type of data:', type(data))
print('lengths:', len(data))
print('shape of elements:', data[0].shape)
print('n_atoms:', feat.topology.n_atoms)

100%|██████████| 742/742 [08:28<00:00,  1.46it/s]                                 


type of data: <class 'list'>
lengths: 742
shape of elements: (80085, 3)
n_atoms: 82


In [12]:
save_dir = 'omega_data'
data_object = Data(data)
data_object.save(f"{save_dir}/data.pkl")
print(f"Data save at: {save_dir}/data.pkl")

Data save at: omega_data/data.pkl


## Combine the omega into one large traj, and plot them

In [2]:
class Data:
    def __init__(self, array_list):
        self.array_list = array_list

    def save(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.array_list, f)

    @classmethod
    def load(cls, filename):
        with open(filename, 'rb') as f:
            array_list = pickle.load(f)
        return cls(array_list)

In [3]:
save_dir = 'omega_data'
omega=Data.load(f"{save_dir}/data.pkl").array_list # This is an (716, x, 3) array
combined_omega = np.vstack(omega)
combined_omega = (combined_omega + 90.0) % 360.0 - 90.0 # Adjust omega values into the range (-90, 270)

print(f"The shape of combined_omega is {combined_omega.shape}")
print(combined_omega[:5])

The shape of combined_omega is (49053457, 3)
[[186.34894     8.187271  158.7122   ]
 [149.942      -5.4726562 180.65887  ]
 [162.98587     2.9294662 189.42859  ]
 [156.76376   -25.824036  160.43533  ]
 [173.60818   -13.742081  199.69666  ]]


In [4]:
nsnaps = combined_omega[1]
print(nsnaps)

[149.942      -5.4726562 180.65887  ]


In [ ]:
start = 0
nsnaps = combined_omega.shape[0]
t = np.arange(start, nsnaps) * 0.01 # Time axis in ns
step = 10000
downsampled_omega = combined_omega[start:nsnaps:step, :]  # Downsampled omega data

residues_to_plot = [0, 1, 2]
num_residues = len(residues_to_plot)

# Set default font sizes globally
plt.rc('font', size=16)         # controls default text sizes
plt.rc('axes', titlesize=18)    # fontsize of the axes title
plt.rc('axes', labelsize=16)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)   # fontsize of the tick labels
plt.rc('ytick', labelsize=14)   # fontsize of the tick labels
plt.rc('legend', fontsize=18)   # legend fontsize

plt.figure(figsize=(10, 18))
for panel, residue in enumerate(residues_to_plot, 1):
    plt.subplot(num_residues, 1, panel)
    plt.plot(t, combined_omega[start:nsnaps, residue], '.', ms=0.5, color='red')
    plt.xlabel('time (ns)')
    plt.ylabel(f'$\\omega$ {residue}°')
    plt.title(f'Omega angles vs Time for Plain MD in 19AE1-4-A')

plt.tight_layout()

# Ensure the directory exists
outfilename = f'omega_data/Omega_angles_vs_Time_Plain_MD_in_19AE1-4-A.png'
plt.savefig(outfilename, transparent=True, dpi=600)
print(f"Saved at: {outfilename}")
#plt.show()

## Clustering

In [5]:
regularspace_estimator = RegularSpace(
    dmin=170,        # minimum distance between cluster centers, 180 degree
    max_centers=8,   # maximum number of cluster centers
    n_jobs=8,
)

In [6]:
# Fit the omegas data into model 
clustering = regularspace_estimator.fit(combined_omega).fetch_model()
assignments = clustering.transform(combined_omega)

print(clustering.cluster_centers)
print(f"Number of cluster centers: {len(clustering.cluster_centers)}")

[[186.34894     8.187271  158.7122   ]
 [-32.18568    -6.301346  181.85834  ]
 [ -5.1141434  17.413452  -26.175514 ]
 [196.90045   -39.36172   -14.8786545]
 [185.08801   182.7413    184.36853  ]
 [-61.47925   195.46259   192.48914  ]
 [166.84967   180.02225    -0.6526947]
 [-84.398125  180.83795    -0.689682 ]]
Number of cluster centers: 8


In [11]:
## Assign each cluster center into cistrans 
cluster_center = clustering.cluster_centers
#print(cluster_center)

# Convert each value to "cis" or "trans"
cis_trans_labels = np.where(cluster_center < 90, "cis", "trans")

# Print results
for i, (center, labels) in enumerate(zip(cluster_center, cis_trans_labels)):
    print(f"Cluster {i}: {center} → {labels}")


Cluster 0: [186.34894    8.187271 158.7122  ] → ['trans' 'cis' 'trans']
Cluster 1: [-32.18568   -6.301346 181.85834 ] → ['cis' 'cis' 'trans']
Cluster 2: [ -5.1141434  17.413452  -26.175514 ] → ['cis' 'cis' 'cis']
Cluster 3: [196.90045   -39.36172   -14.8786545] → ['trans' 'cis' 'cis']
Cluster 4: [185.08801 182.7413  184.36853] → ['trans' 'trans' 'trans']
Cluster 5: [-61.47925 195.46259 192.48914] → ['cis' 'trans' 'trans']
Cluster 6: [166.84967   180.02225    -0.6526947] → ['trans' 'trans' 'cis']
Cluster 7: [-84.398125 180.83795   -0.689682] → ['cis' 'trans' 'cis']


In [12]:
# count the number of each cluter 
counts = Counter(assignments)
#print(counts)

# Sort the counts 
sorted_counts = dict(sorted(counts.items()))
total_counts = sum(counts.values())
cluster = []

for label, frequency in sorted_counts.items():    
    precentage = (frequency / total_counts) * 100 
    cluster.append(precentage)
    print(f"Assignment {label}: {frequency}, {precentage:.4g}%")

outfile = 'clusters_probability.dat'
np.savetxt(outfile, cluster)

Assignment 0: 10585775, 21.58%
Assignment 1: 3560556, 7.259%
Assignment 2: 4587224, 9.351%
Assignment 3: 8955984, 18.26%
Assignment 4: 7292070, 14.87%
Assignment 5: 3125676, 6.372%
Assignment 6: 7681963, 15.66%
Assignment 7: 3264209, 6.654%


#### Plot the cluster 

In [ ]:
fig = plt.figure(figsize=(6, 10))

# Set the display range
x_min, x_max = -90, 270
y_min, y_max = -90, 270
z_min, z_max = -90, 270

# Viewing angles
elevation = 25  # Example elevation angle
azimuth = 65    # Example azimuthal angle

# Create a 3D subplot for cluster centers
ax1 = fig.add_subplot(211, projection='3d')
ax1.set_title('Cluster centers')
ax1.scatter(combined_omega[:, 0], combined_omega[:, 1], combined_omega[:, 2], c=assignments[:], marker='.', alpha=0.05,cmap='tab20',label='STEPs')
ax1.scatter(clustering.cluster_centers[:, 0], clustering.cluster_centers[:, 1], clustering.cluster_centers[:, 2], c='k', marker='o', alpha=1, label='Clustering Center')
ax1.set_xlim(x_min, x_max)
ax1.set_ylim(y_min, y_max)
ax1.set_zlim(z_min, z_max)
ax1.view_init(elev=elevation, azim=azimuth)  # Set the viewing angle

# Label each cluster center
for i, (x, y, z) in enumerate(clustering.cluster_centers):
    ax1.text(x, y, z, f'Cluster {i}', color='red')

# Create a 3D subplot for assignments
ax2 = fig.add_subplot(212, projection='3d')
ax2.set_title('Assignments of data to centers')
ax2.scatter(combined_omega[:, 0], combined_omega[:, 1], combined_omega[:, 2], c=assignments[:], marker='.', alpha=0.3 , cmap='tab20', label='STEPs')
ax2.set_xlim(x_min, x_max)
ax2.set_ylim(y_min, y_max)
ax2.set_zlim(z_min, z_max)
ax2.view_init(elev=elevation, azim=azimuth)  # Set the viewing angle

# Setting the legend
ax1.legend(loc='upper right', bbox_to_anchor=(2.3, 1.0), facecolor='white', edgecolor='k')
ax2.legend(loc='upper right', bbox_to_anchor=(2.3, 1.0), facecolor='white', edgecolor='k')

# Adding axis labels
ax1.set_xlabel(r'$\omega_0$')
ax1.set_ylabel(r'$\omega_1$')
ax1.set_zlabel(r'$\omega_2$')

ax2.set_xlabel(r'$\omega_0$')
ax2.set_ylabel(r'$\omega_1$')
ax2.set_zlabel(r'$\omega_2$')

# Adjust spacing between subplots
#plt.subplots_adjust(hspace=0.5)  # Increase the horizontal space between subplots

# Adjust layout
plt.tight_layout()

fig.suptitle('Cluster centers and assignments directly after K-mean++ Initialization of 19AE1-4-A')
fig.savefig('omega_data/cluster_centers_and_assignments.png',transparent=True, dpi=600, bbox_inches='tight')
plt.show()

/var/folders/2g/yfyfpy7d3n3fwl_23ytkftvm0000gn/T/ipykernel_23618/1469998254.py:52: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  plt.tight_layout()


KeyboardInterrupt: 